# Assignment

In this assignment we will explore the building blocks required to create a contracting-expanding convolutional neural network (CNN) to perform kidney segmentation on CT.

This tutorial is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found at: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 89 kB 3.4 MB/s 
     |████████████████████████████████| 596 kB 30.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [24]:
import numpy as np, pandas as pd
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers
from jarvis.train import datasets
import tensorflow as tf

# Data

The data used in this tutorial will consist of kidney tumor CT exams derived from the Kidney Tumor Segmentation Challenge (KiTS). More information about he KiTS Challenge can be found here: https://kits21.kits-challenge.org/. In this exercise, we will use this dataset to derive a model for slice-by-slice kidney segmentation. The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/ct_kits`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset

In [3]:
# --- Download dataset
datasets.download(name='ct/kits')

# --- Prepare generators and model inputs
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d-bin', custom_layers=True)

[ 2022-05-03 21:44:27 ] [====================] 100.000% : Extracting archive (0000818 / 0000818) 

# Training

In this assignment we will train a standard 2D U-Net for kidney segmentation. In addition to the baseline U-Net architecture, at minimum **one** of the following modifications must be implemented:

* modification of the contracting / expanding backbone (e.g., incorporating ResNet, Inception, SENet, etc)
* modification of the skip connection to include residual connections
* modification of the skip connection to include additional convolution operations

You are also **encouraged** to try different permuations and customizations to achieve optimal validation accuracy.

### Define backbone model

Feel free to use the `lambda` helper functions as demonstrated in the tutorial.

In [25]:
# ---- kwargs dic, lambda, se func, dsc func

kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'
}

conv = lambda x, filters, strides : layers.Conv3D(filters = filters, strides = strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides = 1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides = (1, 2, 2))))

tran = lambda x, filters, strides : layers.Conv3DTranspose(filters = filters, strides = strides, **kwargs)(x)
tran2 = lambda filters, x: relu(norm(tran(x, filters, strides = (1,2 ,2))))

concat = lambda a, b : layers.Concatenate()([a, b])

def se(layer):
  
  sqz = layers.AveragePooling3D((1, layer.shape[2], layer.shape[3]))(layer)
  cha = int(layer.shape[-1]/4)
  exc = layers.Conv3D(filters = cha, kernel_size = 1, activation = 'relu')(sqz)
  sca = layers.Conv3D(filters = layer.shape[-1], kernel_size = 1, activation = 'sigmoid')(exc)

  return layer * sca

def calc_dsc(y_true, y_pred, c=1):

  true = y_true[..., 0] == c
  pred = tf.math.argmax(y_pred, axis =-1) == c

  A = tf.math.count_nonzero(true & pred) * 2
  B = tf.math.count_nonzero(true) + tf.math.count_nonzero(pred)

  return tf.math.divide_no_nan(
      tf.cast(A, tf.float32),
      tf.cast(B, tf.float32)
  )

In [26]:
# --- Define input
x = Input(shape=(None, 96, 96, 1), dtype='float32')

# --- Define model

l1 = conv1(8, x)
l1 = se(l1)

l2 = conv1(16, conv2(16, l1))
l2 = se(l2)

l3 = conv1(32, conv2(32,l2))
l3 = se(l3)

l4 = conv1(48, conv2(48, l3))
l4 = se(l4)

l5 = conv1(64, conv2(64, l4))
l5 = se(l5)

l6 = tran2(48, l5)

l7 = tran2(32, conv1(48, concat(l4, l6))) 

l8 = tran2(16, conv1(32, concat(l3, l7)))

l9 = tran2(8, conv1(16, concat(l2, l8)))

l10 = conv1(8, l9)




# --- Define logits
logits = layers.Conv3D(filters = 2, **kwargs)(l10)

# --- Create model
backbone = Model(inputs=x, outputs=logits)

In [27]:
backbone.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, None, 96, 9  0           []                               
                                6, 1)]                                                            
                                                                                                  
 conv3d_72 (Conv3D)             (None, None, 96, 96  80          ['input_6[0][0]']                
                                , 8)                                                              
                                                                                                  
 batch_normalization_49 (BatchN  (None, None, 96, 96  32         ['conv3d_72[0][0]']              
 ormalization)                  , 8)                                                        

### Define training model

In [28]:
# --- Define inputs
inputs = {
    'dat': Input(shape= (None, 96, 96, 1), name = 'dat'),
    'lbl': Input(shape = (None, 96, 96, 1), name = 'lbl')}

# --- Define model
logits = backbone(inputs['dat'])

# --- Define loss
sce = losses.SparseCategoricalCrossentropy(from_logits = True)
loss = sce(y_true=inputs['lbl'], y_pred = logits)

# --- Define metric
dsc = calc_dsc(y_true = inputs['lbl'], y_pred = logits)

Now, we are ready to create the `training` model and add the corresponding loss and accuracy tensors:

In [29]:
# --- Create model
training = Model(inputs=inputs, outputs={'logits': logits, 'loss' : loss, 'dsc' : dsc})

# --- Add loss
training.add_loss(loss)

# --- Add metric
training.add_metric(dsc, name = 'dsc')

### Compiling

Once the `training` model has been created, use the following to define an optimizer and compile:

In [30]:
# --- Define optimizer 
optimizer = optimizers.Adam(learning_rate = 2e-4)

# --- Compile model
training.compile(optimizer = optimizer)

The model is now compiled and ready for training!

### In-memory data

To speed up training, consider loading all your model data into RAM memory:

In [31]:
# --- Load data into memory for faster training
client.load_data_in_memory()

[ 2022-05-03 22:13:19 ] [====================] 100.000% : Iterating | 000402    

### Train the model

In [32]:
training.fit(
    x=gen_train, 
    steps_per_epoch=100, 
    epochs=20,
    validation_data=gen_valid,
    validation_steps=100,
    validation_freq=5)

Epoch 1/20
100/100 [==============================] - 18s 28ms/step - loss: 0.6829 - dsc: 0.2298
Epoch 2/20
100/100 [==============================] - 3s 28ms/step - loss: 0.3549 - dsc: 0.2450
Epoch 3/20
100/100 [==============================] - 3s 27ms/step - loss: 0.1990 - dsc: 0.7986
Epoch 4/20
100/100 [==============================] - 3s 27ms/step - loss: 0.1257 - dsc: 0.8743
Epoch 5/20
100/100 [==============================] - 5s 51ms/step - loss: 0.0873 - dsc: 0.8995 - val_loss: 0.0871 - val_dsc: 0.8527
Epoch 6/20
100/100 [==============================] - 3s 27ms/step - loss: 0.0658 - dsc: 0.9163
Epoch 7/20
100/100 [==============================] - 3s 28ms/step - loss: 0.0531 - dsc: 0.9281
Epoch 8/20
100/100 [==============================] - 3s 27ms/step - loss: 0.0472 - dsc: 0.9294
Epoch 9/20
100/100 [==============================] - 3s 27ms/step - loss: 0.0421 - dsc: 0.9336
Epoch 10/20
100/100 [==============================] - 5s 46ms/step - loss: 0.0371 - dsc: 0.9407 -

# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `model.predict(...)` on the data aggregated via a test generator.

The Dice score values should be calculated for the foreground class (tumor); the Dice score for background does not need to be evaluated. As discussed in the tutorial, accuracy is determined on a patient by patient (volume by volume) basis, so please calculate the Dice score values on the entire 3D volume (not slice-by-slice).

### Performance

The following minimum performance metrics must be met for full credit:

* kidney: mean Dice score > 0.80

In [35]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

dsc = []

for x, _ in test_valid:

    # pass

    outputs = training.predict(x)

    dsc.append(outputs['dsc'])

dsc = np.array(dsc)

[ 2022-05-03 22:16:20 ] [====================] 100.000% : Iterating | 000081    

**Note**: this cell is used only to check for model performance prior to submission. It will not be graded. Once submitted, your model will be benchmarked against the (same) validation cohort to determine final algorithm performance and grade. If your evaluation code above is correct the algorithm accuracy should match and you can confident that you will recieve full credit for the assignment. Once you are satisfied with your model, proceed to submission of your assignment below.

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort Dice score statistics. There is no need to submit training performance accuracy. As in the tutorial, ensure that the tumor Dice score calculation column is present in the `*.csv` file.

In [36]:
# --- Create *.csv
df = pd.DataFrame(index = np.arange(dsc.size))
df['Dice']= dsc
# --- Serialize *.csv
print(df['Dice'].mean())
df.to_csv('./dice_results.csv')

0.9323396101410006


# Submission

Use the following line to save your model for submission (in Google Colab this should save your model file into your personal Google Drive):

In [37]:
# --- Serialize a model
backbone.save('./wjhan_model.hdf5')

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.